### IMPORT MODULES

In [1]:
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd
import requests
import re

## create an engine to connect to the db

In [2]:
engine = create_engine('sqlite:///data/raw_data_project_m1.db')

## query to select the table we are interested in

In [3]:
pd.read_sql_query("select * from poll_info", engine).head(5)

,uuid,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,I know something about it,I would not vote,None of the above,None of the above,None of the above
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working
2,83127080-da3d-0133-c74f-0a81e8b09a82,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...
3,15626d40-db13-0133-ea5c-0a81e8b09a82,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above
4,24954a70-db98-0133-4a64-0a81e8b09a82,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...


In [4]:
pd.read_sql_query("select * from career_info", engine).head(5)

,uuid,dem_education_level,dem_full_time_job,normalized_job_code
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,no,no,None
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,861a9b9151e11362eb3c77ca914172d0
2,83127080-da3d-0133-c74f-0a81e8b09a82,None,no,None
3,15626d40-db13-0133-ea5c-0a81e8b09a82,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c
4,24954a70-db98-0133-4a64-0a81e8b09a82,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f


In [5]:
pd.read_sql_query("select * from country_info", engine)

,uuid,country_code,rural
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,rural
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,urban
2,83127080-da3d-0133-c74f-0a81e8b09a82,AT,urban
3,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,rural
4,24954a70-db98-0133-4a64-0a81e8b09a82,AT,urban
...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,SK,urban
9645,39f989f0-db52-0133-8482-0a81e8b09a82,SK,urban
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,SK,urban
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,SK,urban


group by the column we are interested in

In [6]:
pd.read_sql_query("select * from country_info group by rural", engine)

,uuid,country_code,rural
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,rural
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,urban


## Changing rural column

We need to change Non-Rural/city to urban and Country/countryside to rural

In [7]:
pd.read_sql_query("select replace (replace (replace (replace (rural, 'Country', 'rural'), 'countryside', 'rural'), 'Non-Rural', 'urban'), 'city', 'urban') AS 'rural' from country_info", engine)


,rural
0,rural
1,urban
2,urban
3,rural
4,urban
...,...
9644,urban
9645,urban
9646,urban
9647,urban


### GETTING THE COLUMNS WE ARE INTERESTED IN

In [8]:
final_db = pd.read_sql_query("SELECT country_info.uuid, country_info.country_code, country_info.rural, career_info.normalized_job_code FROM country_info JOIN career_info ON career_info.uuid = country_info.uuid",engine)
final_db

,uuid,country_code,rural,question_bbi_2016wave4_basicincome_awareness,normalized_job_code
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,rural,I know something about it,None
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,urban,I understand it fully,861a9b9151e11362eb3c77ca914172d0
2,83127080-da3d-0133-c74f-0a81e8b09a82,AT,urban,I have heard just a little about it,None
3,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,rural,I have heard just a little about it,049a3f3a2b5f85cb2971ba77ad66e10c
4,24954a70-db98-0133-4a64-0a81e8b09a82,AT,urban,I have heard just a little about it,f4b2fb1aa40f661488e2782b6d57ad2f
...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,SK,urban,I understand it fully,847165cfda6b1dc82ae22b967da8af2f
9645,39f989f0-db52-0133-8482-0a81e8b09a82,SK,urban,I have never heard of it,a4d5b8b38f9513825d0d94a981ebe962
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,SK,urban,I have heard just a little about it,None
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,SK,urban,I understand it fully,775190277a849cba701b306a7b374c0a


In [9]:
type(final_db)

pandas.core.frame.DataFrame

# API WORK

## Connecting to API (related job)

In [10]:
response = requests.get('http://api.dataatwork.org/v1/jobs/861a9b9151e11362eb3c77ca914172d0/related_jobs')
response.content

b'{"related_job_titles": [{"uuid": "97540260779e3ed95254cec83475ed07", "title": "Applications Systems Analyst", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "861a9b9151e11362eb3c77ca914172d0", "title": "Automatic Data Processing Planner", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "50c2c974472fde19aed2e47b35691577", "title": "Computer Analyst", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "9d7dda6d7915650c681128203a896506", "title": "Computer Analyst Supervisor", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "5ab17bbc6fd0008e239d2d4be22f6a6c", "title": "Computer Methods Analyst", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "fcaf4aac737e53ad24b9f32f08b8740f", "title": "Computer or Data Processing Systems Consultant", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "9c2953b636a8cc64e87087594868ed58", "title": "Computer Systems Analyst", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid

## Getting json

## We need to transform the data into a json "type"

In [11]:
transform = {"uuid": "97540260779e3ed95254cec83475ed07", "title": "Applications Systems Analyst", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "861a9b9151e11362eb3c77ca914172d0", "title": "Automatic Data Processing Planner", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "50c2c974472fde19aed2e47b35691577", "title": "Computer Analyst", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "9d7dda6d7915650c681128203a896506", "title": "Computer Analyst Supervisor", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "5ab17bbc6fd0008e239d2d4be22f6a6c", "title": "Computer Methods Analyst", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "fcaf4aac737e53ad24b9f32f08b8740f", "title": "Computer or Data Processing Systems Consultant", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "9c2953b636a8cc64e87087594868ed58", "title": "Computer Systems Analyst", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "bf2e6241e484f7b95b6f56e947d3ca53", "title": "Computer Systems Consultant", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "e79df6bc40d3640f7f1c0d45298f5989", "title": "Computer Systems Design Analyst", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "d803646eb5eb0353451a13c8f6fc7f9e", "title": "Computer Systems Designer", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "721f8a5289ff95f866272ea0941b6459", "title": "Cross-Enterprise Integrator", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "894ca61c22718617c7e2404f582153da", "title": "Data Processing Consultant", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "e7b3e1c6a32c7247cd4857e4c3574165", "title": "Data Processing Systems Analyst", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "b24ece3300d2f8ab34f9553e531183d6", "title": "Data Processing Systems Project Planner", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "70150ea5cc2c4a0dcb9d3dc238835f2e", "title": "Digital Computer Systems Analyst", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "747b0eb1a51900e9c200409022732366", "title": "E-Business Specialist", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "efa63c7ddecde82a992dcc9c2175d0ab", "title": "Engineering Systems Analyst", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "f7e387ed7f3b36dc521a46160ca91954", "title": "Functional Analyst", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "1749cdbc2468d6628bcd833ff0ac1a11", "title": "Healthcare Applications Analyst", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "7cf9a601e023e14526d10cc77605e401", "title": "Information Systems Analyst (ISA)", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "968efdcdc4f635e64f6bf9978c9d52e9", "title": "Information Systems Architect", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "c5c0c419a2ae9b68c9244bb6ac995073", "title": "Information Systems Consultant", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "8fd2c0b754c45385dd8b68c34f71c505", "title": "Information Technology Generalist (IT Generalist)", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "f271021b74461e6226374cbc7d465941", "title": "Information Technology Specialist (ITS)", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "d5a901898ceaeadb622d28d73d83e3cb", "title": "Information Technology Systems Manager", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "cdf220c51580a4b7cdc548dadd630ab9", "title": "Internet E-Commerce Specialist", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "63b451f06614bd51fd53a70879afdad5", "title": "IT Business Analyst (Information Technology Business Analyst)", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "7604157a983a06a7908cdd0698e4a0a2", "title": "Management Information System Analyst (MIS Analyst)", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "2e8ad38bd092daaaf8955ba735c19440", "title": "Programming Specialist", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "060721c98fdfc207dbc8d1d8e37a7232", "title": "Scientific Systems Analyst", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "7c1a76c4c31058476f255402242d9927", "title": "Software Consultant", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "12f112b5a3fba1af653835f51c257330", "title": "Systems Administration Analyst", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "e5a8bd5e00b6c3ec703eaec9a28d80bd", "title": "Systems Analysis Information Technology Specialist", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "6debe7648b28e668b47b135cdc002ef5", "title": "Systems Architect", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "2808fb0c20b6bd4ba7cb423360122372", "title": "Health Systems Analyst", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "a135111342b3da84c967eba58e8c494e", "title": "Information Systems Auditor", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "90974ee1e64d53f4ab01d863fd105cdb", "title": "Information Systems Planner", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "e0a65e148d67764a6fcba9c1cd5ec024", "title": "Information Technology Auditor (IT Auditor)", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "90082e1c828d92eccbb9c4b7a145b06b", "title": "Applications Analyst", "parent_uuid": "0148f61d4227497728ce33490843d056"}, {"uuid": "cf81121543f08d762c89add012cdd707", "title": "Business Analyst", "parent_uuid": "0148f61d4227497728ce33490843d056"}
    

In [12]:
df_related_jobs = pd.json_normalize(transform)
df_related_jobs

,uuid,title,parent_uuid
0,97540260779e3ed95254cec83475ed07,Applications Systems Analyst,0148f61d4227497728ce33490843d056
1,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,0148f61d4227497728ce33490843d056
2,50c2c974472fde19aed2e47b35691577,Computer Analyst,0148f61d4227497728ce33490843d056
3,9d7dda6d7915650c681128203a896506,Computer Analyst Supervisor,0148f61d4227497728ce33490843d056
4,5ab17bbc6fd0008e239d2d4be22f6a6c,Computer Methods Analyst,0148f61d4227497728ce33490843d056
5,fcaf4aac737e53ad24b9f32f08b8740f,Computer or Data Processing Systems Consultant,0148f61d4227497728ce33490843d056
6,9c2953b636a8cc64e87087594868ed58,Computer Systems Analyst,0148f61d4227497728ce33490843d056
7,bf2e6241e484f7b95b6f56e947d3ca53,Computer Systems Consultant,0148f61d4227497728ce33490843d056
8,e79df6bc40d3640f7f1c0d45298f5989,Computer Systems Design Analyst,0148f61d4227497728ce33490843d056
9,d803646eb5eb0353451a13c8f6fc7f9e,Computer Systems Designer,0148f61d4227497728ce33490843d056


## RENAME UUID COLUMN

In [13]:
final_api_df = df_related_jobs.rename(columns={'uuid': 'normalized_job_code'})
final_api_df

,normalized_job_code,title,parent_uuid
0,97540260779e3ed95254cec83475ed07,Applications Systems Analyst,0148f61d4227497728ce33490843d056
1,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,0148f61d4227497728ce33490843d056
2,50c2c974472fde19aed2e47b35691577,Computer Analyst,0148f61d4227497728ce33490843d056
3,9d7dda6d7915650c681128203a896506,Computer Analyst Supervisor,0148f61d4227497728ce33490843d056
4,5ab17bbc6fd0008e239d2d4be22f6a6c,Computer Methods Analyst,0148f61d4227497728ce33490843d056
5,fcaf4aac737e53ad24b9f32f08b8740f,Computer or Data Processing Systems Consultant,0148f61d4227497728ce33490843d056
6,9c2953b636a8cc64e87087594868ed58,Computer Systems Analyst,0148f61d4227497728ce33490843d056
7,bf2e6241e484f7b95b6f56e947d3ca53,Computer Systems Consultant,0148f61d4227497728ce33490843d056
8,e79df6bc40d3640f7f1c0d45298f5989,Computer Systems Design Analyst,0148f61d4227497728ce33490843d056
9,d803646eb5eb0353451a13c8f6fc7f9e,Computer Systems Designer,0148f61d4227497728ce33490843d056


# WEB SCRAPPING

In [14]:
!pip install lxml

In [15]:
income_url  = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
dfs = pd.read_html(income_url)
dfs

[          0     1        2     3            4     5         6     7
 0   Belgium  (BE)   Greece  (EL)    Lithuania  (LT)  Portugal  (PT)
 1  Bulgaria  (BG)    Spain  (ES)   Luxembourg  (LU)   Romania  (RO)
 2   Czechia  (CZ)   France  (FR)      Hungary  (HU)  Slovenia  (SI)
 3   Denmark  (DK)  Croatia  (HR)        Malta  (MT)  Slovakia  (SK)
 4   Germany  (DE)    Italy  (IT)  Netherlands  (NL)   Finland  (FI)
 5   Estonia  (EE)   Cyprus  (CY)      Austria  (AT)    Sweden  (SE)
 6   Ireland  (IE)   Latvia  (LV)       Poland  (PL)       NaN   NaN,
                0     1   2            3     4
 0        Iceland  (IS) NaN       Norway  (NO)
 1  Liechtenstein  (LI) NaN  Switzerland  (CH),
                 0     1   2
 0  United Kingdom  (UK) NaN,
                  0     1
 0       Montenegro  (ME)
 1  North Macedonia  (MK)
 2          Albania  (AL)
 3           Serbia  (RS)
 4           Turkey  (TR),
                         0        1
 0  Bosnia and Herzegovina     (BA)
 1               

In [16]:
together = pd.concat(dfs[0:13], axis=1, ignore_index=True) #axis=1 so we don't have code with country mixed
together

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,Belgium,(BE),Greece,(EL),Lithuania,(LT),Portugal,(PT),Iceland,(IS),...,(AR),NaN,China (except Hong Kong),(CN_X_HK),NaN,Mexico,(MX),NaN,South Africa,(ZA)
1,Bulgaria,(BG),Spain,(ES),Luxembourg,(LU),Romania,(RO),Liechtenstein,(LI),...,(AU),NaN,Hong Kong,(HK),NaN,Nigeria,(NG),NaN,South Korea,(KR)
2,Czechia,(CZ),France,(FR),Hungary,(HU),Slovenia,(SI),NaN,NaN,...,(BR),NaN,India,(IN),NaN,New Zealand,(NZ),NaN,Taiwan,(TW)
3,Denmark,(DK),Croatia,(HR),Malta,(MT),Slovakia,(SK),NaN,NaN,...,(CA),NaN,Japan,(JP),NaN,Singapore,(SG),NaN,United States,(US)
4,Germany,(DE),Italy,(IT),Netherlands,(NL),Finland,(FI),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Estonia,(EE),Cyprus,(CY),Austria,(AT),Sweden,(SE),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Ireland,(IE),Latvia,(LV),Poland,(PL),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Delete columns with 100% null values

In [17]:
together_without_nan = together.dropna(axis=1, how='all')
together_without_nan

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,41,42,44,45,47,48
0,Belgium,(BE),Greece,(EL),Lithuania,(LT),Portugal,(PT),Iceland,(IS),...,Russia,(RU),Argentina,(AR),China (except Hong Kong),(CN_X_HK),Mexico,(MX),South Africa,(ZA)
1,Bulgaria,(BG),Spain,(ES),Luxembourg,(LU),Romania,(RO),Liechtenstein,(LI),...,NaN,NaN,Australia,(AU),Hong Kong,(HK),Nigeria,(NG),South Korea,(KR)
2,Czechia,(CZ),France,(FR),Hungary,(HU),Slovenia,(SI),NaN,NaN,...,NaN,NaN,Brazil,(BR),India,(IN),New Zealand,(NZ),Taiwan,(TW)
3,Denmark,(DK),Croatia,(HR),Malta,(MT),Slovakia,(SK),NaN,NaN,...,NaN,NaN,Canada,(CA),Japan,(JP),Singapore,(SG),United States,(US)
4,Germany,(DE),Italy,(IT),Netherlands,(NL),Finland,(FI),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Estonia,(EE),Cyprus,(CY),Austria,(AT),Sweden,(SE),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Ireland,(IE),Latvia,(LV),Poland,(PL),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# usamos df.melt para poner las columnas debajo
country_melt = together[[0,2,4,6,8,11,14,16,18,20,23, 26,28,30,32,34,36,38,41,44,47]].melt(value_name='Country')['Country']
code_melt = together[[1,3,5,7,9,12,15,17,19,21,24, 27,29,31,33,35,37,39,42,45,48]].melt(value_name='country_code')['country_code']
country = pd.concat([together, country_melt], axis=1)
code = pd.concat([country, code_melt], axis=1)
country_code_df = code[['Country', 'country_code']]
country_code_df

,Country,country_code
0,Belgium,(BE)
1,Bulgaria,(BG)
2,Czechia,(CZ)
3,Denmark,(DK)
4,Germany,(DE)
...,...,...
142,Taiwan,(TW)
143,United States,(US)
144,NaN,NaN
145,NaN,NaN


## drop null rows and correcting Code column

In [19]:
good_df = country_code_df.dropna(axis=0, thresh=1) #thresh = 1 so we need minimum 2 NaN values in a row

In [20]:
def clean_parenthesis (code_column):
    return re.sub('[()]', '', code_column)

good_df['country_code'] = good_df['country_code'].astype(str).apply(clean_parenthesis)
good_df

<ipython-input-20-77b3f5b1be47>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_df['country_code'] = good_df['country_code'].astype(str).apply(clean_parenthesis)


,Country,country_code
0,Belgium,BE
1,Bulgaria,BG
2,Czechia,CZ
3,Denmark,DK
4,Germany,DE
...,...,...
136,Singapore,SG
140,South Africa,ZA
141,South Korea,KR
142,Taiwan,TW


# MERGE THE DATAFRAMES

### Join final_db and api on normalized_job_code

In [21]:
db_api = pd.merge(final_db, final_api_df, on=['normalized_job_code'], how='outer') 
#outer para que coja datos de ambas tablas aunque puede que haya datos que no existan, por un null de una de las tablas. Hacer inner si nos interesan aquellos que si tengan código de trabajo
db_api

,uuid,country_code,rural,question_bbi_2016wave4_basicincome_awareness,normalized_job_code,title,parent_uuid
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,rural,I know something about it,None,NaN,NaN
1,83127080-da3d-0133-c74f-0a81e8b09a82,AT,urban,I have heard just a little about it,None,NaN,NaN
2,b50dbb80-da53-0133-8956-0a81e8b09a82,AT,urban,I know something about it,None,NaN,NaN
3,9949c4c0-da5f-0133-c832-0a81e8b09a82,AT,urban,I understand it fully,None,NaN,NaN
4,69f1f400-dc5f-0133-ad9b-0a81e8b09a82,AT,urban,I know something about it,None,NaN,NaN
...,...,...,...,...,...,...,...
9679,NaN,NaN,NaN,NaN,a135111342b3da84c967eba58e8c494e,Information Systems Auditor,0148f61d4227497728ce33490843d056
9680,NaN,NaN,NaN,NaN,90974ee1e64d53f4ab01d863fd105cdb,Information Systems Planner,0148f61d4227497728ce33490843d056
9681,NaN,NaN,NaN,NaN,e0a65e148d67764a6fcba9c1cd5ec024,Information Technology Auditor (IT Auditor),0148f61d4227497728ce33490843d056
9682,NaN,NaN,NaN,NaN,90082e1c828d92eccbb9c4b7a145b06b,Applications Analyst,0148f61d4227497728ce33490843d056


In [22]:
project1_df = pd.merge(db_api, good_df, on=['country_code'], how='outer') 
project1_df

,uuid,country_code,rural,question_bbi_2016wave4_basicincome_awareness,normalized_job_code,title,parent_uuid,Country
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,rural,I know something about it,None,NaN,NaN,Austria
1,83127080-da3d-0133-c74f-0a81e8b09a82,AT,urban,I have heard just a little about it,None,NaN,NaN,Austria
2,b50dbb80-da53-0133-8956-0a81e8b09a82,AT,urban,I know something about it,None,NaN,NaN,Austria
3,9949c4c0-da5f-0133-c832-0a81e8b09a82,AT,urban,I understand it fully,None,NaN,NaN,Austria
4,69f1f400-dc5f-0133-ad9b-0a81e8b09a82,AT,urban,I know something about it,None,NaN,NaN,Austria
...,...,...,...,...,...,...,...,...
9729,NaN,SG,NaN,NaN,NaN,NaN,NaN,Singapore
9730,NaN,ZA,NaN,NaN,NaN,NaN,NaN,South Africa
9731,NaN,KR,NaN,NaN,NaN,NaN,NaN,South Korea
9732,NaN,TW,NaN,NaN,NaN,NaN,NaN,Taiwan


### Dropping nan rows (uuid and title)

In [25]:
project1_df.dropna(subset=['uuid', 'normalized_job_code', 'title'], inplace=True)
project1_df.reset_index(inplace=True)
project1_df.head(10)

,index,uuid,country_code,rural,question_bbi_2016wave4_basicincome_awareness,normalized_job_code,title,parent_uuid,Country
0,59,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,urban,I understand it fully,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,0148f61d4227497728ce33490843d056,Austria
1,60,cf24ed60-da3f-0133-0034-0a81e8b09a82,AT,rural,I understand it fully,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,0148f61d4227497728ce33490843d056,Austria
2,76,68eace50-db46-0133-97f7-0a81e8b09a82,AT,urban,I have heard just a little about it,fcaf4aac737e53ad24b9f32f08b8740f,Computer or Data Processing Systems Consultant,0148f61d4227497728ce33490843d056,Austria
3,77,11c61a10-de3d-0133-6041-0a81e8b09a82,AT,urban,I understand it fully,fcaf4aac737e53ad24b9f32f08b8740f,Computer or Data Processing Systems Consultant,0148f61d4227497728ce33490843d056,Austria
4,78,5eaadd90-da5a-0133-3d70-0a81e8b09a82,AT,rural,I understand it fully,fcaf4aac737e53ad24b9f32f08b8740f,Computer or Data Processing Systems Consultant,0148f61d4227497728ce33490843d056,Austria
5,94,5441c1d0-d9a6-0133-55bc-0a81e8b09a82,AT,urban,I know something about it,b24ece3300d2f8ab34f9553e531183d6,Data Processing Systems Project Planner,0148f61d4227497728ce33490843d056,Austria
6,209,9ffc7390-dba5-0133-259b-0a81e8b09a82,BE,rural,I understand it fully,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,0148f61d4227497728ce33490843d056,Belgium
7,210,a6361230-da58-0133-cd5a-0a81e8b09a82,BE,urban,I understand it fully,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,0148f61d4227497728ce33490843d056,Belgium
8,227,c9337360-db95-0133-d531-0a81e8b09a82,BE,urban,I know something about it,b24ece3300d2f8ab34f9553e531183d6,Data Processing Systems Project Planner,0148f61d4227497728ce33490843d056,Belgium
9,228,9b1600f0-da4b-0133-27da-0a81e8b09a82,BE,urban,I have heard just a little about it,b24ece3300d2f8ab34f9553e531183d6,Data Processing Systems Project Planner,0148f61d4227497728ce33490843d056,Belgium


In [57]:
project = project1_df[['Country', 'title', 'rural']].groupby(['Country','title', 'rural']).size().reset_index(name='Quantity') 
#I use size instead of agg or directly.count `cause with size I can count 'directly'
project

,Country,title,rural,Quantity
0,Austria,Automatic Data Processing Planner,rural,1
1,Austria,Automatic Data Processing Planner,urban,1
2,Austria,Computer or Data Processing Systems Consultant,rural,1
3,Austria,Computer or Data Processing Systems Consultant,urban,2
4,Austria,Data Processing Systems Project Planner,urban,1
...,...,...,...,...
104,Spain,Data Processing Systems Analyst,urban,9
105,Spain,Data Processing Systems Project Planner,rural,1
106,Spain,Data Processing Systems Project Planner,urban,7
107,Sweden,Automatic Data Processing Planner,urban,4


### Create a new column with percentage

In [78]:
project['Percentage'] = ((project['Quantity']/project['Quantity'].count())*100).round(2)
project.sample(10)

,Country,title,rural,Quantity,Percentage
21,Denmark,Automatic Data Processing Planner,rural,1,0.92
107,Sweden,Automatic Data Processing Planner,urban,4,3.67
43,Germany,Data Processing Systems Analyst,rural,4,3.67
102,Spain,Data Processing Consultant,urban,5,4.59
96,Slovenia,Data Processing Systems Analyst,rural,1,0.92
42,Germany,Data Processing Consultant,urban,8,7.34
50,Ireland,Data Processing Consultant,urban,2,1.83
75,Poland,Data Processing Systems Project Planner,urban,3,2.75
84,Romania,Data Processing Consultant,urban,2,1.83
94,Slovakia,Data Processing Systems Project Planner,rural,2,1.83
